# Пример выполнения тестового задания SQL

## Описание БД

![title](pics/DB_test1.jpg)

<b>Задание 1</b>
<p>Получите список всех сотрудников (таблица Employees), отсортированную по алфавиту.
Что нужно вывести: все поля из таблицы employees.</p>

In [ ]:
SELECT *
FROM Employees
ORDER BY last_nm
    , first_nm
    , middle_nm

<b>Задание 2</b>
<p>Получите список сотрудников с названиями их позиций (position_nm), актуальных на 10.10.2020.</p>

In [ ]:
SELECT a.last_nm
    , a.first_nm
    , a.middle_nm
    , b.position_nm
FROM Employees a
    JOIN Job_history b
        ON a.position_id=b.position id
WHERE b.valid_to_ddtm >= ‘2020.10.10’ 
	AND b.valid_from_ddtm <= ‘2020.10.10’
--(этим условием пользоваться, если таблица empolyees содержит список всех сотрудников, в т.ч. и тех, что уже уволились)
--Если таблица еmpoloyees не содержит записей об уволенных сотрудниках, достаточно условия WHERE a.hire_dt <= ‘2020.10.10’ и тип связи тогда лучше использовать Left join 

<b>Задание 3</b>
<p>Получите список сотрудников, у которых ЗП (salary) от 20 000 до 50 000, актуальных на текущий момент</p>

In [ ]:
SELECT a.last_nm
    , a.first_nm
    , a.middle_nm
    , b.salary
    , a.hire_dt
FROM Employees a
    JOIN Job_history b
        ON a.position_id=b.position_id
WHERE b.salary between 20000 and 50000
AND b.valid_to_dttm >= CURRENT_DATE

<b>Задание 4</b>
<p>Получите список сотрудников в формате: «Иванова - Наталья – Юрьевна» (ФИО должно быть прописано в одном столбике, разделение ‘-’)</p>

In [ ]:
SELECT CONCAT(last_nm, ‘-‘, first_nm, ‘-‘, middle_nm) AS fio
    , birth_dt
FROM Employees

<b>Задание 5</b>
<p>Получите количество звонков (таблица Calls) по дням, период с 01.10.2020 по текущий день</p>

In [ ]:
SELECT date_trunc('day', start_dttm) AS date
    	, count(call_id) as cnt_calls
FROM Calls
WHERE start_dttm >= ‘2020.10.01’
-- можно такое условие, если важна ссылка на текущую дату WHERE start_dttm between ‘2020.10.01’ AND CURRENT_DATE
GROUP BY date_trunc('day', start_dttm)
--ORDER BY date_trunc('day', start_dttm) – сортировка не обязательна, но, на мой взгляд, так более упорядоченно

<b>Задание 6</b>
<p>Вывести %% дозвона для каждого дня (%% дозвона – это доля принятых звонков (dozv_flg = 1) от всех поступивших звонков (dozv_flg =1 or dozv_flg = 0)), период с 01.10.2020 по текущий день</p>

In [ ]:
SELECT date_trunc('day', start_dttm) AS date
    	, sum(dozv_flg)::int/count(dozv_flg)::float*100 AS sla
FROM Calls
WHERE start_dttm >= ‘2020.10.01’
-- можно такое условие, если важна ссылка на текущую дату WHERE start_dttm BETWEEN ‘2020.10.01’ AND CURRENT_DATE
GROUP BY date_trunc('day', start_dttm)
--ORDER BY date_trunc('day', start_dttm) – сортировка не обязательна, но, на мой взгляд, так более упорядоченно

<b>Задание 7</b>
<p>Получите длительность самого длинного разговора, и самого короткого за 28.10.2020 (start_dttm – начало звонка, end_dttm - окончание звонка)</p>

In [ ]:
SELECT max(end_dttm - start_dttm) AS max_call_time
    , min(end_dttm - start_dttm) AS min_call_time
FROM call_length
WHERE date_trunc('day', start_dttm) = ‘2020.10.28’

<b>Задание 8</b>
<p>Получите информацию по всем звонкам, в какой системе они были обработаны (таблица System)</p>

In [ ]:
SELECT a.call_id
    , a.start_dttm
    , a.end_dttm
    , b.system_name
FROM calls a
    LEFT JOIN system b
        ON a.system_id = b.system_id

<b>Задание 9</b>
<p>Получите информацию по клиентам (таблица Customers), которые звонили и не дозвонились 23.10.2020.</p>

In [ ]:
SELECT date_trunc('day', a.start_dttm) AS date
    , b.last_nm
    , b.first_nm
    , b.middle_nm
FROM calls a
    JOIN customers b
        ON a.customer_id = b.customer_id
WHERE date_trunc('day', a.start_dttm) = ‘2020.10.23’
	AND a.dozv_flg = 0

<b>Задание 10</b>
<p>Получите информацию по продуктам, которые были открыты у клиентов во время звонка с 05.10.2020 по текущий день</p>

In [ ]:
SELECT b.last_nm
    , b.first_nm
    , b.middle_nm date_trunc
    , ('day', a.start_dttm) AS date
    , b.open_dt
    , b.product_nm
FROM calls a
    JOIN customers b
        ON a.customer_id = b.customer_id
        AND b.open_dt BETWEEN a.start_dttm AND a.end_dttm
WHERE date_trunc('day', a.start_dttm) >= ‘2020.10.05’
-- ИЛИ WHERE date_trunc('day', a.start_dttm) BETWEEN ‘2020.10.05’ AND CURRENT_DATE

<b>Задание 11</b>
<p>С помощью оконных функций выведите информацию по тем звонкам, на которых данный employee_id уже не первый раз общается с данным customer_id</p>

In [ ]:
SELECT customer_id
    , employee_id
    , start_dttm
    , end_dttm
FROM
    (SELECT customer_id
        , employee_id
        , start_dttm
        , end_dttm
        , row_number() over (partition by customer_id, employee_id order by start_dttm) AS rn_customer_employee
    FROM calls
    WHERE dozv_flg = 1 – я бы ввела это условие, чтобы исключить непринятые звонки
    ) t
WHERE rn_customer_employee > 1